In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
import sklearn.utils
import sklearn.metrics
import iisignature
import torch
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import List, Optional, Dict, Set, Callable, Any
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets
import sigkernel
import scipy
from scipy.interpolate import interp1d
from numba import njit
import pickle

from signature import streams_to_sigs, transform_stream
from conformance import BaseclassConformanceScore, stream_to_torch
from kernels import linear_kernel_gram, rbf_kernel_gram, poly_kernel_gram
from kernels import pairwise_kernel_gram, integral_kernel_gram, sig_kernel_gram

# tslearn datasets (equal length)

* equal length (in time) UCR_UEA multivariate time series 

In [3]:
# #run experiments

# experiments = run_tslearn_experiments(
#     dataset_names = [
#         #'ArticularyWordRecognition', 
#         #'BasicMotions', 
#         #'Cricket',
#          ##########'ERing', #cant find dataset
#         'Libras', 
#         #'NATOPS', 
#         #'RacketSports',     
#         #'FingerMovements',
#         #'Heartbeat',
#         #'SelfRegulationSCP1', 
#         #'UWaveGestureLibrary'
#         ],
#     kernel_names = [
#         "linear",
#         #"rbf",
#         #"poly",
#         #"gak",
#         #"truncated sig",
#         #"truncated sig rbf",
#         #"truncated sig poly",
#         #"signature pde",
#         #"signature pde rbf",
#         #"signature pde poly",
#         #"integral linear",
#         #"integral rbf",
#         #"integral poly",
#         ],
#         verbose=True
#         )


# def print_experiment_results(experiments, round_digits=5):
#     for dataset_name, results in experiments.items():
#         #Dataset:
#         print("\nStart Dataset {dataset_name} results:", dataset_name)
#         print_dataset_stats(results["num_classes"], results["dim"], 
#                             results["ts_length"], results["N_train"], 
#                             results["N_test"])

#         #Results for each kernel:
#         for kernel_name, scores in results["results"].items():
#             print("\nKernel:", kernel_name)
#             scores = np.mean(scores, axis=2)
#             print("Conformance AUC:", round(scores[0, 0], round_digits))
#             print("Mahalanobis AUC:", round(scores[1, 0], round_digits))
#             print("Conformance PR AUC:", round(scores[0, 1], round_digits))
#             print("Mahalanobis PR AUC:", round(scores[1, 1], round_digits))

#         print("\nEnd Dataset {dataset_name} results\n\n\n")
        
# print_experiment_results(experiments)

In [4]:
#comments

# Kernel: truncated sig
# Conformance AUC: 0.90757
# Mahalanobis AUC: 0.90728
# Conformance PR AUC: 0.99236
# Mahalanobis PR AUC: 0.99235

# Kernel: linear
# Conformance AUC: 0.94646
# Mahalanobis AUC: 0.93737
# Conformance PR AUC: 0.99506
# Mahalanobis PR AUC: 0.99445

# Kernel: rbf
# Conformance AUC: 0.7671
# Mahalanobis AUC: 0.08571
# Conformance PR AUC: 0.95905
# Mahalanobis PR AUC: 0.84199

# Kernel: integral linear
# Conformance AUC: 0.94686
# Mahalanobis AUC: 0.93849
# Conformance PR AUC: 0.99512
# Mahalanobis PR AUC: 0.99451

# End Dataset {dataset_name} results

## Find all tslearn datasets

In [5]:
# _datasets = [
#             'ArticularyWordRecognition', 
#             'BasicMotions', 
#             'Cricket',
#             #'ERing',
#             'Libras', 
#             'NATOPS', 
#             'RacketSports',     
#             'FingerMovements',
#             'Heartbeat',
#             'SelfRegulationSCP1', 
#             'UWaveGestureLibrary'
#             ]

# import tslearn
# UCR_UEA_datasets = tslearn.datasets.UCR_UEA_datasets()

# for dataset_name in UCR_UEA_datasets.list_multivariate_datasets():
# #for dataset_name in _datasets:
#     print("Dataset:", dataset_name)
#     dataset = UCR_UEA_datasets.load_dataset(dataset_name)
#     if dataset[0] is not None:
#         X_train, y_train, X_test, y_test = dataset
#         num_classes = len(np.unique(y_train))
#         N_train, T, d = X_train.shape
#         N_test, _, _  = X_test.shape
        
#         print("Number of Classes:", num_classes)
#         print("Dimension of path:", d)
#         print("Length:", T)
#         print("Train Size, Test Size", N_train, N_test)
#         print()
#     else:
#         print("No dataset found")
#         print()

#yes
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: AtrialFibrillation
# No dataset found

#yes
# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: CharacterTrajectories
# No dataset found

#yes
# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: DuckDuckGeese
# No dataset found

# Dataset: EigenWorms
# Number of Classes: 5
# Dimension of path: 6
# Length: 17984
# Train Size, Test Size 128 131

#why not
# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train Size, Test Size 137 138

#longLength
# Dataset: EthanolConcentration
# Number of Classes: 4
# Dimension of path: 3
# Length: 1751
# Train Size, Test Size 261 263

# Dataset: ERing
# No dataset found

#big
# Dataset: FaceDetection
# Number of Classes: 2
# Dimension of path: 144
# Length: 62
# Train Size, Test Size 5890 3524

#yes
# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

#why not, maybe big length
# Dataset: HandMovementDirection
# Number of Classes: 4
# Dimension of path: 10
# Length: 400
# Train Size, Test Size 160 74

#smallTrain
# Dataset: Handwriting
# Number of Classes: 26
# Dimension of path: 3
# Length: 152
# Train Size, Test Size 150 850

#yes
# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

#big
# Dataset: InsectWingbeat
# Number of Classes: 10
# Dimension of path: 200
# Length: 22
# Train Size, Test Size 25000 25000

# Dataset: JapaneseVowels
# No dataset found

#yes
# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

#TODO I SHOULD INCLUDE
# Dataset: LSST
# Number of Classes: 14
# Dimension of path: 6
# Length: 36
# Train Size, Test Size 2459 2466

#length
# Dataset: MotorImagery
# Number of Classes: 2
# Dimension of path: 64
# Length: 3000
# Train Size, Test Size 278 100

#yes
# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

#TODO NOT TSLEARN. LENGTH WRONG
# Dataset: PenDigits
# Number of Classes: 10
# Dimension of path: 2
# Length: 8
# Train Size, Test Size 7494 3498

#highDim
# Dataset: PEMS-SF
# Number of Classes: 7
# Dimension of path: 963
# Length: 144
# Train Size, Test Size 267 173

#dim=1, big length
# Dataset: Phoneme
# Number of Classes: 39
# Dimension of path: 1
# Length: 1024
# Train Size, Test Size 214 1896

#yes
# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

#yes
# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: SelfRegulationSCP2
# Number of Classes: 2
# Dimension of path: 7
# Length: 1152
# Train Size, Test Size 200 180

# Dataset: SpokenArabicDigits
# No dataset found

#long, also very small set
# Dataset: StandWalkJump
# Number of Classes: 3
# Dimension of path: 4
# Length: 2500
# Train Size, Test Size 12 15

#yes
# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320


# Cross Validation code (for anomaly detection)

In [7]:
from cross_validation import cv_tslearn

cv_best_models = cv_tslearn(
    dataset_names = [
        #'ArticularyWordRecognition', 
        #'BasicMotions', 
        #'Cricket',
         ##########'ERing', #cant find dataset
        'Libras', 
        #'NATOPS', 
        #'RacketSports',     
        #'FingerMovements',
        #'Heartbeat',
        #'SelfRegulationSCP1', 
        #'UWaveGestureLibrary'
        ],
    kernel_names = [
        "linear",
        #"rbf",
        #"poly",
        #"gak",
        #"truncated sig",
        #"truncated sig rbf",
        #"truncated sig poly",
        #"signature pde",
        #"signature pde rbf",
        #"signature pde poly",
        #"integral linear",
        #"integral rbf",
        #"integral poly",
        ],
        k=5,
        n_repeats=10,
        n_jobs_repeats=4
        )

print(cv_best_models)

Dataset: Libras


Label for linear: 100%|██████████| 15/15 [00:06<00:00,  2.18it/s]

Time taken for kernel linear: 6.8868889808654785 seconds
Time taken for dataset Libras: 6.905257940292358 seconds



{'Libras': {'kernel_results': {'linear': {'1': {'kernel_name': 'linear', 'normal_class_label': '1', 'threshold': 1, 'CV_train_auc': 0.746825396825397, 'auc_params': array([0.7468254]), 'auc_thresh': 0.746825396825397}, '10': {'kernel_name': 'linear', 'normal_class_label': '10', 'threshold': 1, 'CV_train_auc': 0.8356261022927691, 'auc_params': array([0.8356261]), 'auc_thresh': 0.8356261022927691}, '11': {'kernel_name': 'linear', 'normal_class_label': '11', 'threshold': 1, 'CV_train_auc': 0.9572222222222222, 'auc_params': array([0.95722222]), 'auc_thresh': 0.9572222222222222}, '12': {'kernel_name': 'linear', 'normal_class_label': '12', 'threshold': 1, 'CV_train_auc': 0.8649250440917109, 'auc_params': array([0.86492504]), 'auc_thresh': 0.8649250440917109}, '13': {'kernel_name': 'linear', 'normal_class_label': '13', 'threshold': 1, 'CV_train_auc': 0.9071516754850089, 'auc_pa